In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import datetime

In [2]:
data_file_name = 'face_data.npz'
test_file_name = 'test_data.npz'

batch_size = 128

def init_weights(shape, name):
    """
    initialize weight
    shape: shape of weight
    name: weight name
    각각의 Filter를 만들어주는 함수
    """
    with tf.device('/gpu:0'):
        return tf.Variable(tf.random_normal(shape, stddev = 0.005), name = name)

def init_bias(shape, name):
    """
    initialize bias
    bias를 만들어주는 함수
    """
    with tf.device('/gpu:0'):
        return tf.Variable(tf.constant(0.1, shape = shape))

def model(X, w1, w2, w3, w4, wo, b2, b3, b4, bo, p_keep_conv, p_keep_hidden):
    """
    create model
    X: first input
    w1, w2, w3, wo: weights for each layer
    b2, b3, bo: bias for each layer
    p_keep_conv, p_keep_hidden: constant for dropout
    
    모델생성 함수.
    총 3개의 Conv layers와 3개의 Max-pool layers, 2개의 FC layers로 구성
    Conv1 - Pooling1 - Conv2 - Pooling2 - Conv3 - Pooling3 - FC1 - FC2
    로 구성되어있고 각 풀링 이후와 첫번째 FC 이후에 dropout을 적용
    """
    
    with tf.device('/gpu:0'):
#         dw = tf.nn.max_pool(X, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') 

        # First Conv layer. l1a shape = (?, 128, 128, 32)
        with tf.name_scope("layer1"):
            l1a = tf.nn.relu(tf.nn.conv2d(X, w1, strides = [1, 1, 1, 1], padding = 'SAME'))
            # First Max-pool layer. l1 shape = (?, 64, 64, 32)
            l1 = tf.nn.max_pool(l1a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')        
            l1 = tf.nn.dropout(l1, p_keep_conv)

        with tf.name_scope("layer2"):
            # l2a shape = (?, 64, 64, 64)
            l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides = [1, 1, 1, 1], padding = 'SAME') + b2)
            # l2 shape = (?, 32, 32, 64)
            l2 = tf.nn.max_pool(l2a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            l2 = tf.nn.dropout(l2, p_keep_conv)

        with tf.name_scope("layer3"):
            # l2a shape = (?, 64, 64, 64)
            l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides = [1, 1, 1, 1], padding = 'SAME') + b3)
            # l2 shape = (?, 32, 32, 64)
            l3 = tf.nn.max_pool(l3a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
            l3 = tf.nn.dropout(l3, p_keep_conv)

        with tf.name_scope("layer4"):
            # reshape to ( , 3072)
            l4 = tf.nn.relu(tf.matmul(l3, w4) + b4)
            l4 = tf.nn.dropout(l4, p_keep_hidden)
            l4 = tf.matmul(l4, wo) + bo
            return l4

In [7]:
if os.path.isfile(data_file_name):
    file = np.load(data_file_name)
    data = file['data']
    labels = file['sex_labels']
else:
    print("No data file.")
    sys.exit()
    
if os.path.isfile(test_file_name):
    file = np.load(test_file_name)
    td = file['data']
    tl = file['sex_labels']
else:
    print("No test file")
    sys.exit()

print("data load success")

print(data.shape)
print(td.shape)

data load success
(1785, 128, 128, 1)
(166, 128, 128, 1)


In [4]:
tf.reset_default_graph()

# data, labels 를 위한 placeholder
X = tf.placeholder("float", [None, 128, 128, 1], name = "X")
Y = tf.placeholder("float", [None, 2], name = "Y")


w1 = init_weights([3, 3, 1, 32], "weight1") 
w2 = init_weights([3, 3, 32, 64], "weight2")
w3 = init_weights([3, 3, 64, 128], "weight3")
w4 = init_weights([16*16*128, 512], "weight4")
wo = init_weights([512, 2], "weight_out")

b2 = init_bias([64], "bias2")
b3 = init_bias([128], "bias3")
b4 = init_bias([512], "bias4")
bo = init_bias([2], "bias_out")

tf.summary.histogram("weight1_summ", w1)
tf.summary.histogram("weight2_summ", w2)
tf.summary.histogram("bias2_summ", b2)
tf.summary.histogram("weight3_summ", w3)
tf.summary.histogram("bias3_summ", b3)
tf.summary.histogram("weight4_summ", w4)
tf.summary.histogram("bias4_summ", b4)
tf.summary.histogram("weight_out_summ", wo)
tf.summary.histogram("bias_out_summ", bo)

# dropout 위한 placeholder
p_keep_conv = tf.placeholder("float", name = "p_keep_conv")
p_keep_hidden = tf.placeholder("float", name = "p_keep_hidden")

M = model(X, w1, w2, w3, w4, wo, b2, b3, b4, bo, p_keep_conv, p_keep_hidden)

with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = M, labels = Y))
    train_op = tf.train.RMSPropOptimizer(0.00001, 0.9).minimize(cost)
    tf.summary.scalar("cost", cost)

with tf.name_scope("accuarcy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(M, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float"))
    tf.summary.scalar("accuracy", acc_op)
    
tf.add_to_collection('vars', w1)
tf.add_to_collection('vars', w2)
tf.add_to_collection('vars', b2)
tf.add_to_collection('vars', w3)
tf.add_to_collection('vars', b3)
tf.add_to_collection('vars', w4)
tf.add_to_collection('vars', b4)
tf.add_to_collection('vars', wo)
tf.add_to_collection('vars', bo)

In [5]:
save_path = "./models/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
with tf.Session(config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))) as sess:
    
    writer = tf.summary.FileWriter('./logs/face2', sess.graph)
    merged = tf.summary.merge_all()
    
    tf.global_variables_initializer().run()
    
    saver = tf.train.Saver()
    
    print(datetime.datetime.now().time())
    
    for i in range(450):
        training_batch = zip(range(0, len(data), batch_size), range(batch_size, len(data)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict = {X: data[start:end], Y: labels[start:end],
                                           p_keep_conv: 0.8, p_keep_hidden: 0.5})
        
        summary, acc = sess.run([merged, acc_op], feed_dict = {X: td, Y: tl, p_keep_conv: 1.0, p_keep_hidden: 1.0})
        
        writer.add_summary(summary, i)
        print(i, acc)
    
    saved_path = saver.save(sess, os.path.join(save_path, 'face_recog_2.ckpt'))
    print("Checkpoint saved at ", saved_path)
    print(datetime.datetime.now().time())
    sess.close()

14:46:07.387983
0 0.921687
1 0.921687
2 0.921687
3 0.921687
4 0.921687
5 0.921687
6 0.921687
7 0.921687
8 0.0783133
9 0.0783132
10 0.0783132
11 0.0783133
12 0.0783133
13 0.0783133
14 0.0783132
15 0.0783133
16 0.0783132
17 0.0783132
18 0.0783133
19 0.0783133
20 0.0783132
21 0.0783133
22 0.0783133
23 0.0783132
24 0.0783133
25 0.0783132
26 0.0783132
27 0.0783132
28 0.0783132
29 0.0783133
30 0.0783132
31 0.0783133
32 0.0783133
33 0.0783133
34 0.0783133
35 0.0783133
36 0.0783133
37 0.0783133
38 0.0783132
39 0.0783133
40 0.0783132
41 0.0783132
42 0.0783132
43 0.0783132
44 0.0783132
45 0.0783132
46 0.0783133
47 0.0783132
48 0.0783132
49 0.0783132
50 0.0783132
51 0.0783132
52 0.0783132
53 0.0783133
54 0.0783132
55 0.0783133
56 0.0843373
57 0.0903614
58 0.10241
59 0.126506
60 0.162651
61 0.198795
62 0.222892
63 0.295181
64 0.337349
65 0.385542
66 0.415663
67 0.439759
68 0.463855
69 0.487952
70 0.512048
71 0.53012
72 0.566265
73 0.572289
74 0.572289
75 0.584337
76 0.60241
77 0.608434
78 0.626506